## Testing playlist genre

In [ ]:
# IMPORTANTE!!! Esta celda tarda en ejecutarse mucho tiempo

# Selecciono la lista final a extraer completa
list_toextract_artist = test_list_artist

# Función para extraerlos, en lotes de 50 porque es el máximo que te deja la API de Spotify
def extract_artist(artist_list):
    count_artist= 0
    data = ['test11111111', 'tes111111', 'test111111', 0, 0]
    df_final = pd.DataFrame([data], columns=["id", "name", "genres", 'followers', 'popularity'])
    
    API_LIMIT = 50
    search_list = []

    for artist in artist_list:
        count_artist += 1
        search_results = sp.search(q = 'artist:"{}"'.format(artist), type="artist",  limit=API_LIMIT)
        
        count_oneartist= 0 # Se añade un contador par quedarnos sólo con la primera búsqueda que he comprobado que siempre es la buena.
        for t in search_results['artists']['items']:
            count_oneartist += 1
            if count_oneartist ==2:
                break
            else:
                search_list += [[t["id"], t["name"], t["genres"], t['followers']['total'], t['popularity']]]
        
        df_search = pd.DataFrame(search_list, columns=["id", "name", "genres", 'followers', 'popularity'])
        
        df_final = df_final.append(df_search) # El problema de esto es que crea duplicados pero no nos afecta en lo que necesitamos
    print('Artist count:', count_artist)  
    return df_final

# Extracción de los datos
Time1 = datetime.now()
df_genres = extract_artist(list_toextract_artist)
Time2 = datetime.now()
print("Tiempo ejecución:", Time2 -Time1)

print(df_genres.shape)


# Tarda en ejecutarse dependiendo del número de artistas, menos de 1 minuto
# Creamos una copia de la columna genres para mantenerla y la trasnformo a strings
df_genres['genres_together'] = df_genres['genres'].copy()
df_genres['genres_together'] = [','.join(i) if isinstance(i, list) else i for i in df_genres['genres_together']]

# Como la columna genres es una lista con los diferentes géneros, he preferido separarlo en columnas y mantener ciertas columnas

df_genres2 = pd.concat([df_genres['genres'].apply(pd.Series), df_genres[['genres_together',
                                                                         'id',
                                                                         "name",
                                                                         'followers',
                                                                         'popularity']]], axis = 1)
# Quitamos los artistas duplicados
df_genres3 = df_genres2.drop_duplicates(df_genres2.columns[~df_genres2.columns.isin(['id'])])

#Renombramos columnas
df_genres4 = df_genres3.rename(columns={0:'genre1',
                   1:'genre2',
                   2:'genre3',
                   3:'genre4',
                   4:'genre5',
                   5:'genre6',
                   6:'genre7',
                   7:'genre8',
                   8:'genre9',
                   9:'genre10',
                   'genres_together':'genres_together',
                   'id':'artist_id',
                   'name':'artist',
                   'followers':'artist_followers',
                   'popularity':'artist_popularity'}).copy()
print(df_genres4.shape)
print(len(df_genres4['artist'].unique()))
df_genres4.head(10)

In [ ]:
df_genres4.isnull().sum(axis = 0)

In [ ]:
# Export songs list
data_glo = pd.merge(data_features_final, df_genres4, on=['artist'], how='inner')
data_glo.head(5)
data_glo.to_csv("data_ana_playlist_songs.csv", sep = ',')

# Para extraer canciones y artistas



In [ ]:
# IMPORTANTE, Tarda alrededor de 30min en sacarlo todo, por eso ejecutamos antes una muestra sólo

#FALTA JUNTAR ARTISTAS CON NOMBRES Y VER CÓMO LO JUNTAMOS EN AL DTAFRAME FINAL
# Sacar las canciones de las listas para tenerlas todas juntas
# Tarda bastante tiempo, alrededor de XX

playlists_extract = playlists

# Para sacar canciones
Time1 = datetime.now()
songs = []
for uri in playlists_extract['uri']:

        lenght = sp.playlist_tracks(uri)['total']
        for i in range(0,lenght, 50):
            init_data = sp.playlist_tracks(uri, limit=50, offset=i)
            try:
                for k in init_data['items']:
                    songs.append(k['track']["id"])
            except:
                pass

Time2 = datetime.now()
print("Tiempo ejecución:", Time2 -Time1)


# Para sacar artistas
Time1 = datetime.now()
artists = []
for uri in playlists_sample['uri']:

        lenght = sp.playlist_tracks(uri)['total']
        for i in range(0,lenght, 50):
            init_data = sp.playlist_tracks(uri, limit=50, offset=i)
            try:
                for k in init_data['items']:
                    artists.append(k['track']['artists'][0]['name'])
            except:
                pass

Time2 = datetime.now()
print("Tiempo ejecución:", Time2 -Time1)

print(len(songs))